In [1]:
"""
Online XGBoost Multi-Fidelity Parameter Optimization
- Truly online: decides after each run whether to do low or high fidelity
- XGBoost surrogate (better than GP for 20+ samples)  
- Multi-fidelity: 30-day for exploration, 180-day for verification
- Adaptive: runs high-fidelity on promising candidates as we go
"""

import numpy as np
import pickle
import os
from scipy.stats import qmc
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from qg_model import QGTwoLayerModel
from scipy.ndimage import uniform_filter

# ============================================================================
# COLOR PRINTING
# ============================================================================

class Colors:
    """ANSI color codes for terminal output"""
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    
    @staticmethod
    def green(text):
        return f"{Colors.OKGREEN}{text}{Colors.ENDC}"
    
    @staticmethod
    def red(text):
        return f"{Colors.FAIL}{text}{Colors.ENDC}"
    
    @staticmethod
    def blue(text):
        return f"{Colors.OKBLUE}{text}{Colors.ENDC}"
    
    @staticmethod
    def cyan(text):
        return f"{Colors.OKCYAN}{text}{Colors.ENDC}"
    
    @staticmethod
    def yellow(text):
        return f"{Colors.WARNING}{text}{Colors.ENDC}"
    
    @staticmethod
    def bold(text):
        return f"{Colors.BOLD}{text}{Colors.ENDC}"
    
    @staticmethod
    def success(text):
        return f"{Colors.OKGREEN}{Colors.BOLD}✓ {text}{Colors.ENDC}"
    
    @staticmethod
    def error(text):
        return f"{Colors.FAIL}{Colors.BOLD}✗ {text}{Colors.ENDC}"
    
    @staticmethod
    def star(text):
        return f"{Colors.WARNING}{Colors.BOLD}★ {text}{Colors.ENDC}"

# ============================================================================
# PARAMETER BOUNDS
# ============================================================================

PARAM_BOUNDS = {
    'viscosity_scale': {
        'bounds': (0.5, 5.0),
        'type': 'linear',
        'description': 'Multiplies hyperviscosity coefficient',
    },
    'drag_scale': {
        'bounds': (0.5, 3.0),
        'type': 'linear',
        'description': 'Multiplies Ekman drag coefficient',
    },
    'eddy_diffusivity': {
        'bounds': (1e3, 1e5),
        'type': 'log',
        'description': 'Additional biharmonic diffusion (m²/s)',
    },
    'smagorinsky_coeff': {
        'bounds': (0.0, 0.3),
        'type': 'linear',
        'description': 'Smagorinsky eddy viscosity coefficient',
    },
    'energy_correction': {
        'bounds': (-0.01, 0.01),
        'type': 'linear',
        'description': 'Energy backscatter correction',
    },
    'enstrophy_correction': {
        'bounds': (0.0, 1e-6),
        'type': 'log',
        'description': 'Enstrophy dissipation rate',
    },
}

PARAM_NAMES = list(PARAM_BOUNDS.keys())
N_PARAMS = len(PARAM_NAMES)

# ============================================================================
# LATIN HYPERCUBE SAMPLING
# ============================================================================

def generate_latin_hypercube_samples(n_samples, bounds_dict):
    """Generate Latin Hypercube samples"""
    n_params = len(bounds_dict)
    sampler = qmc.LatinHypercube(d=n_params, seed=42)
    unit_samples = sampler.random(n=n_samples)
    
    samples = np.zeros_like(unit_samples)
    for i, (param_name, param_info) in enumerate(bounds_dict.items()):
        lower, upper = param_info['bounds']
        
        if param_info['type'] == 'log':
            log_lower = np.log10(lower) if lower > 0 else -10
            log_upper = np.log10(upper)
            samples[:, i] = 10 ** (unit_samples[:, i] * (log_upper - log_lower) + log_lower)
        else:
            samples[:, i] = unit_samples[:, i] * (upper - lower) + lower
    
    return samples

# ============================================================================
# SIMULATION RUNNER
# ============================================================================

def run_lowres_with_params(params_array, config_base, highres_results, 
                           sim_days=180, fidelity='high'):
    """
    Run low-res simulation with given parameters
    
    Multi-Fidelity Strategy:
    - Low-fidelity (30 days): Compare spinup dynamics to first 30 days of high-res
    - High-fidelity (180 days): Compare equilibrium to last 30 days of high-res
    
    This ensures we're comparing matching temporal dynamics.
    
    Parameters:
    -----------
    params_array : np.ndarray
        Parameter values
    config_base : dict
        Base configuration
    highres_results : dict
        High-res results (full 180 days, pre-computed)
    sim_days : int
        Not used - determined by fidelity
    fidelity : str
        'low' (30 days) or 'high' (180 days)
    
    Returns:
    --------
    loss : float
        Combined loss (NaN if failed)
    results : dict or None
        Simulation results
    detailed_outputs : dict or None
        Detailed outputs
    """
    from main_comparison import run_simulation
    
    # Set simulation length based on fidelity
    if fidelity == 'low':
        actual_sim_days = 30
        n_days_avg = 30  # Average over all 30 days
    else:
        actual_sim_days = 180
        n_days_avg = 30  # Average over last 30 days
    
    config = config_base.copy()
    config['subgrid_params'] = {PARAM_NAMES[i]: float(params_array[i]) for i in range(N_PARAMS)}
    
    print(f"\n{'='*70}")
    print(f"Testing parameters [{fidelity.upper()}-fidelity: {actual_sim_days} days]:")
    for param_name, val in config['subgrid_params'].items():
        print(f"  {param_name}: {val:.6e}")
    
    try:
        results = run_simulation(config, sim_days=actual_sim_days, save_interval_hours=12)
        
        # Compute loss - high-res is already 180 days, we use last 30 for comparison
        loss, detailed_outputs = compute_loss(results, highres_results, 
                                             n_days_avg=n_days_avg, 
                                             return_fields=True)
        
        if not np.isfinite(loss):
            print(f"  ✗ Loss is not finite: {loss}")
            return np.nan, None, None
        
        print(f"  Loss: {loss:.6f} (averaged over {n_days_avg} days)")
        return loss, results, detailed_outputs
        
    except Exception as e:
        print(Colors.error(f"Simulation failed: {e}"))
        import traceback
        traceback.print_exc()
        return np.nan, None, None

# ============================================================================
# LOSS COMPUTATION
# ============================================================================

def compute_loss(lowres_results, highres_results, n_days_avg=30, return_fields=False):
    """
    Compute loss - Multi-fidelity version
    
    Strategy:
    - For 30-day low-res runs: Compare to FIRST 30 days of high-res (spinup dynamics)
    - For 180-day high-res runs: Compare to LAST 30 days of high-res (equilibrium)
    
    This ensures fair comparison at matching simulation times.
    """
    
    nx_hr = highres_results['config']['nx']
    ny_hr = highres_results['config']['ny']
    nx_lr = lowres_results['config']['nx']
    ny_lr = lowres_results['config']['ny']
    
    if nx_hr % nx_lr != 0 or ny_hr % ny_lr != 0:
        raise ValueError(f"Grid mismatch")
    
    coarsen_factor_x = nx_hr // nx_lr
    coarsen_factor_y = ny_hr // ny_lr
    
    times_hr = highres_results['times']
    times_lr = lowres_results['times']
    
    # Determine which part of high-res to compare against
    lr_duration = times_lr[-1] - times_lr[0]
    
    if lr_duration <= 35:  # Low-fidelity run (≤30 days + margin)
        # Compare to FIRST 30 days of high-res
        time_start_hr = times_hr[0]
        time_end_hr = time_start_hr + 30
        indices_hr = np.where((times_hr >= time_start_hr) & (times_hr <= time_end_hr))[0]
        comparison_type = "first 30 days (spinup)"
    else:  # High-fidelity run (180 days)
        # Compare to LAST 30 days of high-res (equilibrium)
        time_threshold_hr = times_hr[-1] - 30
        indices_hr = np.where(times_hr >= time_threshold_hr)[0]
        comparison_type = "last 30 days (equilibrium)"
    
    # Low-res: use last n_days_avg days
    time_threshold_lr = times_lr[-1] - n_days_avg
    indices_lr = np.where(times_lr >= time_threshold_lr)[0]
    
    print(f"  Comparing: Low-res last {n_days_avg} days vs High-res {comparison_type}")
    print(f"    High-res: {len(indices_hr)} snapshots")
    print(f"    Low-res:  {len(indices_lr)} snapshots")
    
    q1_hr_avg = np.mean([highres_results['q1_history'][i] for i in indices_hr], axis=0)
    q2_hr_avg = np.mean([highres_results['q2_history'][i] for i in indices_hr], axis=0)
    q1_lr_avg = np.mean([lowres_results['q1_history'][i] for i in indices_lr], axis=0)
    q2_lr_avg = np.mean([lowres_results['q2_history'][i] for i in indices_lr], axis=0)
    
    model_hr = highres_results['model']
    model_lr = lowres_results['model']
    
    psi1_hr_avg, psi2_hr_avg = model_hr.q_to_psi(q1_hr_avg, q2_hr_avg)
    psi1_lr_avg, psi2_lr_avg = model_lr.q_to_psi(q1_lr_avg, q2_lr_avg)
    
    H1, H2 = model_hr.H1, model_hr.H2
    H_total = H1 + H2
    
    q_bt_hr = (H1 * q1_hr_avg + H2 * q2_hr_avg) / H_total
    psi_bt_hr = (H1 * psi1_hr_avg + H2 * psi2_hr_avg) / H_total
    q_bt_lr = (H1 * q1_lr_avg + H2 * q2_lr_avg) / H_total
    psi_bt_lr = (H1 * psi1_lr_avg + H2 * psi2_lr_avg) / H_total
    
    def coarsen(field, factor_x, factor_y):
        filtered = uniform_filter(field, size=(factor_y, factor_x), mode='wrap')
        return filtered[::factor_y, ::factor_x]
    
    q_bt_hr_coarse = coarsen(q_bt_hr, coarsen_factor_x, coarsen_factor_y)
    psi_bt_hr_coarse = coarsen(psi_bt_hr, coarsen_factor_x, coarsen_factor_y)
    
    def nrmse(pred, target):
        mse = np.mean((pred - target)**2)
        std = np.std(target)
        return np.sqrt(mse) / (std + 1e-20)
    
    loss_q_bt = nrmse(q_bt_lr, q_bt_hr_coarse)
    loss_psi_bt = nrmse(psi_bt_lr, psi_bt_hr_coarse)
    
    weight_pv = 0.6
    weight_psi = 0.4
    total_loss = weight_pv * loss_q_bt + weight_psi * loss_psi_bt
    
    if return_fields:
        return total_loss, {
            'q_bt_hr_coarse': q_bt_hr_coarse, 'psi_bt_hr_coarse': psi_bt_hr_coarse,
            'q_bt_lr': q_bt_lr, 'psi_bt_lr': psi_bt_lr,
            'loss_q_bt': loss_q_bt, 'loss_psi_bt': loss_psi_bt,
            'total_loss': total_loss, 'weight_pv': weight_pv, 'weight_psi': weight_psi,
        }
    return total_loss

# ============================================================================
# ONLINE XGBOOST MULTI-FIDELITY OPTIMIZER
# ============================================================================

class OnlineXGBoostOptimizer:
    """
    Online XGBoost Multi-Fidelity Bayesian Optimization
    
    Adaptively decides after each run whether to:
    1. Explore new region (low-fidelity 30-day run)
    2. Verify promising candidate (high-fidelity 180-day run)
    """
    
    def __init__(self, bounds_dict, n_initial_samples=12):
        self.bounds_dict = bounds_dict
        self.n_params = len(bounds_dict)
        self.n_initial_samples = n_initial_samples
        
        # Storage - each sample tracks params, loss, fidelity
        self.samples = []  # List of dicts: {'params', 'loss', 'fidelity', 'detailed'}
        
        self.best_loss = np.inf
        self.best_params = None
        self.best_iteration = -1
        self.iteration = 0
        
        # XGBoost models (separate for each fidelity)
        self.model_low = XGBRegressor(
            n_estimators=100, max_depth=6, learning_rate=0.1,
            subsample=0.8, colsample_bytree=0.8, random_state=42
        )
        
        self.model_high = XGBRegressor(
            n_estimators=100, max_depth=6, learning_rate=0.1,
            subsample=0.8, colsample_bytree=0.8, random_state=42
        )
        
        # Correlation model (predicts high-fidelity from low-fidelity loss)
        self.correlation_model = None
    
    def random_sample(self):
        """Generate random sample within bounds"""
        sample = np.zeros(self.n_params)
        for i, (param_name, param_info) in enumerate(self.bounds_dict.items()):
            lower, upper = param_info['bounds']
            if param_info['type'] == 'log':
                log_lower = np.log10(lower) if lower > 0 else -10
                log_upper = np.log10(upper)
                sample[i] = 10 ** (np.random.uniform(log_lower, log_upper))
            else:
                sample[i] = np.random.uniform(lower, upper)
        return sample
    
    def get_samples_by_fidelity(self, fidelity):
        """Extract samples of specific fidelity"""
        fid_samples = [s for s in self.samples if s['fidelity'] == fidelity and np.isfinite(s['loss'])]
        if not fid_samples:
            return None, None
        X = np.array([s['params'] for s in fid_samples])
        y = np.array([s['loss'] for s in fid_samples])
        return X, y
    
    def fit_models(self):
        """Fit XGBoost models on available data"""
        # Fit low-fidelity model
        X_low, y_low = self.get_samples_by_fidelity('low')
        if X_low is not None and len(X_low) >= 5:
            self.model_low.fit(X_low, y_low)
            print(f"  ✓ Low-fidelity model fitted on {len(X_low)} samples")
        
        # Fit high-fidelity model
        X_high, y_high = self.get_samples_by_fidelity('high')
        if X_high is not None and len(X_high) >= 5:
            self.model_high.fit(X_high, y_high)
            print(f"  ✓ High-fidelity model fitted on {len(X_high)} samples")
            
            # Fit correlation model on common points
            if X_low is not None and len(X_low) >= 3:
                common_low_losses = []
                common_high_losses = []
                
                for s_high in [s for s in self.samples if s['fidelity'] == 'high' and np.isfinite(s['loss'])]:
                    # Find if this param set was also run at low fidelity
                    for s_low in [s for s in self.samples if s['fidelity'] == 'low' and np.isfinite(s['loss'])]:
                        if np.allclose(s_high['params'], s_low['params']):
                            common_low_losses.append(s_low['loss'])
                            common_high_losses.append(s_high['loss'])
                            break
                
                if len(common_low_losses) >= 3:
                    from sklearn.linear_model import Ridge
                    self.correlation_model = Ridge(alpha=0.1)
                    self.correlation_model.fit(
                        np.array(common_low_losses).reshape(-1, 1),
                        np.array(common_high_losses)
                    )
                    print(f"  ✓ Correlation model fitted on {len(common_low_losses)} common points")
    
    def predict_high_from_low(self, params):
        """Predict high-fidelity loss from low-fidelity model + correlation"""
        X_low, y_low = self.get_samples_by_fidelity('low')
        
        if X_low is None or len(X_low) < 5:
            return None
        
        # Predict low-fidelity loss
        params_2d = np.atleast_2d(params)
        pred_low = self.model_low.predict(params_2d)[0]
        
        # Apply correlation if available
        if self.correlation_model is not None:
            pred_high = self.correlation_model.predict([[pred_low]])[0]
            return pred_high
        
        # Otherwise use low-fidelity as proxy (add small penalty for uncertainty)
        return pred_low * 1.1
    
    def decide_next_action(self):
        """
        Decide whether next run should be low or high fidelity
        
        Hybrid Strategy (UPDATED):
        - First 50 iterations: Multi-fidelity (mix low/high for exploration)
        - After iteration 50: ALL high-fidelity (focus on accuracy)
        
        This ensures we don't get stuck in local optima from poor low-fidelity predictions.
        """
        n_total = len(self.samples)
        n_low = len([s for s in self.samples if s['fidelity'] == 'low'])
        n_high = len([s for s in self.samples if s['fidelity'] == 'high'])
        
        # PHASE 2: After 50 iterations, switch to ALL high-fidelity
        if n_total >= 50:
            return 'high', 'pure_high_fidelity_phase'
        
        # PHASE 1: First 50 iterations - multi-fidelity exploration
        
        # Need at least 12 low-fidelity samples for good model
        if n_low < 12:
            return 'low', 'initial_exploration'
        
        # Cap high-fidelity at 20 during exploration phase
        if n_high >= 20:
            return 'low', 'max_high_exploration'
        
        # Every 3rd run after initial phase, do high-fidelity verification
        if (n_low + n_high - 12) % 3 == 0 and n_high < 20:
            return 'high', 'periodic_verification'
        
        return 'low', 'continued_exploration'
    
    def propose_next_sample(self, fidelity):
        """Propose next sample using acquisition function"""
        self.fit_models()
        
        # For high-fidelity, pick best unverified low-fidelity candidate
        if fidelity == 'high':
            X_low, y_low = self.get_samples_by_fidelity('low')
            if X_low is None:
                return self.random_sample()
            
            # Get params already run at high fidelity
            verified_params = [s['params'] for s in self.samples if s['fidelity'] == 'high']
            
            # Find best unverified low-fidelity candidates
            unverified_indices = []
            for i, params in enumerate(X_low):
                is_verified = any(np.allclose(params, vp) for vp in verified_params)
                if not is_verified:
                    unverified_indices.append(i)
            
            if len(unverified_indices) == 0:
                return self.random_sample()
            
            # Return best unverified
            best_unverified_idx = unverified_indices[np.argmin(y_low[unverified_indices])]
            print(f"  ✓ Selecting best unverified (low-fidelity loss: {y_low[best_unverified_idx]:.6f})")
            return X_low[best_unverified_idx]
        
        # For low-fidelity, use UCB acquisition
        X_low, y_low = self.get_samples_by_fidelity('low')
        
        best_acq = -np.inf
        best_x = None
        
        # Random search over candidate points
        for _ in range(500):
            x = self.random_sample()
            
            # UCB acquisition
            if X_low is not None and len(X_low) >= 5:
                pred = self.model_low.predict(np.atleast_2d(x))[0]
                
                # Uncertainty = distance to nearest neighbor
                distances = np.linalg.norm(X_low - x, axis=1)
                uncertainty = np.min(distances) / (np.max(distances) + 1e-10)
                
                acq = -(pred - 2.0 * uncertainty)  # UCB with kappa=2
            else:
                acq = np.random.rand()  # Random if no model
            
            if acq > best_acq:
                best_acq = acq
                best_x = x
        
        if best_x is None:
            best_x = self.random_sample()
        
        print(f"  ✓ Proposed sample with acquisition = {best_acq:.6e}")
        return best_x
    
    def optimize(self, config_base, highres_results, max_iterations=150):
        """
        Run online optimization with hybrid strategy
        
        Phase 1 (iterations 1-50): Multi-fidelity exploration
        Phase 2 (iterations 51+): Pure high-fidelity refinement
        """
        print(f"\n{'='*70}")
        print(f"HYBRID XGBOOST OPTIMIZATION")
        print(f"{'='*70}")
        print("Strategy:")
        print(f"  Phase 1 (iter 1-50):  Multi-fidelity exploration")
        print(f"                        - Low-fidelity (30d) for broad search")
        print(f"                        - Periodic high-fidelity (180d) verification")
        print(f"  Phase 2 (iter 51+):   Pure high-fidelity refinement")
        print(f"                        - ALL runs at 180 days")
        print(f"                        - Focus on finding true optimum")
        print(f"  Max iterations: {max_iterations}")
        print("="*70)
        
        for iteration in range(len(self.samples), max_iterations):
            self.iteration = iteration
            
            print(f"\n{'='*70}")
            print(f"ITERATION {iteration + 1}/{max_iterations}")
            
            # Show phase
            if iteration < 50:
                phase_info = Colors.cyan(f"[PHASE 1: Multi-Fidelity Exploration]")
            else:
                phase_info = Colors.yellow(f"[PHASE 2: High-Fidelity Refinement]")
            print(phase_info)
            print(f"{'='*70}")
            
            # Decide fidelity
            fidelity, reason = self.decide_next_action()
            print(f"Decision: {fidelity.upper()}-fidelity ({reason})")
            
            # Propose parameters
            next_params = self.propose_next_sample(fidelity)
            
            # Run simulation
            loss, results, detailed = run_lowres_with_params(
                next_params, config_base, highres_results, fidelity=fidelity
            )
            
            # Store result
            self.samples.append({
                'params': next_params,
                'loss': loss,
                'fidelity': fidelity,
                'detailed': detailed
            })
            
            # Update best (only from high-fidelity runs)
            if fidelity == 'high' and np.isfinite(loss) and loss < self.best_loss:
                self.best_loss = loss
                self.best_params = next_params.copy()
                self.best_iteration = iteration
                print(Colors.star(f"NEW BEST HIGH-FIDELITY: {Colors.green(f'{loss:.6f}')}"))
            
            # Status
            n_low = len([s for s in self.samples if s['fidelity'] == 'low'])
            n_high = len([s for s in self.samples if s['fidelity'] == 'high'])
            n_low_valid = len([s for s in self.samples if s['fidelity'] == 'low' and np.isfinite(s['loss'])])
            n_high_valid = len([s for s in self.samples if s['fidelity'] == 'high' and np.isfinite(s['loss'])])
            
            print(f"\n  Status:")
            print(f"    Low-fidelity:  {Colors.cyan(f'{n_low_valid}/{n_low}')} successful")
            print(f"    High-fidelity: {Colors.cyan(f'{n_high_valid}/{n_high}')} successful")
            if self.best_loss < np.inf:
                loss_str = Colors.green(f'{self.best_loss:.6f}')
                iter_str = Colors.blue(f'(iteration {self.best_iteration + 1})')
                print(f"    {Colors.bold('Best loss:')} {loss_str} {iter_str}")
            
            # Save progress
            self.save_progress()
        
        return self.get_best_params()
    
    def get_best_params(self):
        """Return best parameters"""
        if self.best_params is None:
            raise ValueError("No valid parameters found!")
        return {PARAM_NAMES[i]: float(self.best_params[i]) for i in range(N_PARAMS)}
    
    def save_progress(self, filename='online_xgboost_mf_progress.pkl'):
        """Save progress"""
        data = {
            'samples': self.samples,
            'best_loss': self.best_loss,
            'best_params': self.best_params,
            'best_iteration': self.best_iteration,
            'iteration': self.iteration,
            'bounds_dict': self.bounds_dict,
            'n_initial_samples': self.n_initial_samples,
        }
        with open(filename, 'wb') as f:
            pickle.dump(data, f)
        print(f"  ✓ Progress saved")
    
    @classmethod
    def load_progress(cls, filename='online_xgboost_mf_progress.pkl'):
        """Load progress"""
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        
        optimizer = cls(data['bounds_dict'], data['n_initial_samples'])
        optimizer.samples = data['samples']
        optimizer.best_loss = data['best_loss']
        optimizer.best_params = data['best_params']
        optimizer.best_iteration = data['best_iteration']
        optimizer.iteration = data['iteration']
        
        n_low = len([s for s in optimizer.samples if s['fidelity'] == 'low'])
        n_high = len([s for s in optimizer.samples if s['fidelity'] == 'high'])
        
        print(f"✓ Loaded checkpoint:")
        print(f"  Total iterations: {len(optimizer.samples)}")
        print(f"  Low-fidelity:  {Colors.cyan(str(n_low))}")
        print(f"  High-fidelity: {Colors.cyan(str(n_high))}")
        loss_str = Colors.green(f'{optimizer.best_loss:.6f}')
        print(f"  {Colors.bold('Best loss:')} {loss_str}")
        
        return optimizer

# ============================================================================
# MAIN
# ============================================================================

def main(checkpoint_file='online_xgboost_mf_progress.pkl', max_iterations=150):
    """Main optimization routine with hybrid strategy"""
    
    print("\n" + "="*70)
    print("HYBRID XGBOOST OPTIMIZATION")
    print("="*70)
    print("\nStrategy:")
    print("  • Iterations 1-50:  Multi-fidelity (30d + 180d mixed)")
    print("  • Iterations 51+:   Pure high-fidelity (all 180d)")
    print("="*70)
    
    # Load high-res (pre-computed for full 180 days)
    if not os.path.exists('highres_results.pkl'):
        print("\n✗ Error: highres_results.pkl not found!")
        print("  Run main_comparison.py first to generate high-res data.")
        return
    
    with open('highres_results.pkl', 'rb') as f:
        highres_results = pickle.load(f)
    print(f"\n✓ Loaded high-res: {highres_results['config']['nx']}x{highres_results['config']['ny']}, 180 days")
    print("  Multi-fidelity comparison strategy:")
    print("    • Low-fidelity (30d):  compares to FIRST 30 days of high-res (spinup)")
    print("    • High-fidelity (180d): compares to LAST 30 days of high-res (equilibrium)")
    print("  This ensures fair temporal matching at each fidelity level")
    
    # Load base config
    from main_comparison import config_lowres
    config_base = config_lowres.copy()
    
    # Load or create optimizer
    if os.path.exists(checkpoint_file):
        print(f"\n✓ Checkpoint found")
        optimizer = OnlineXGBoostOptimizer.load_progress(checkpoint_file)
    else:
        print(f"\n✓ Starting new optimization")
        optimizer = OnlineXGBoostOptimizer(PARAM_BOUNDS, n_initial_samples=12)
    
    # Run optimization
    best_params = optimizer.optimize(config_base, highres_results, max_iterations)
    
    # Results
    print("\n" + "="*70)
    print("OPTIMIZATION COMPLETE")
    print("="*70)
    
    n_low = len([s for s in optimizer.samples if s['fidelity'] == 'low'])
    n_high = len([s for s in optimizer.samples if s['fidelity'] == 'high'])
    
    cost_actual = n_low / 6 + n_high  # 30-day is 1/6 cost of 180-day
    cost_baseline = len(optimizer.samples)  # If all were 180-day
    savings_pct = (1 - cost_actual / cost_baseline) * 100
    
    print(f"\nTotal iterations: {len(optimizer.samples)}")
    print(f"  Low-fidelity (30 days):  {Colors.cyan(str(n_low))}")
    print(f"  High-fidelity (180 days): {Colors.cyan(str(n_high))}")
    cost_str = Colors.yellow(f'~{cost_actual:.1f}x')
    print(f"  Computational cost: {cost_str} vs {cost_baseline}x baseline")
    savings_str = Colors.green(f'{savings_pct:.1f}%')
    print(f"  Savings: {savings_str}")
    loss_str = Colors.green(f'{optimizer.best_loss:.6f}')
    print(f"\n{Colors.bold('Best loss:')} {loss_str}")
    print(f"{Colors.bold('Best parameters:')}")
    for name, val in best_params.items():
        val_str = Colors.cyan(f'{val:.6e}')
        print(f"  {name}: {val_str}")
    
    # Save
    with open('online_xgboost_optimal_params.pkl', 'wb') as f:
        pickle.dump(best_params, f)
    
    with open('online_xgboost_optimal_config.txt', 'w') as f:
        f.write("'subgrid_params': {\n")
        for name, val in best_params.items():
            f.write(f"    '{name}': {val:.6e},\n")
        f.write("}\n")
    
    print("\n✓ Saved: online_xgboost_optimal_params.pkl")
    print("✓ Saved: online_xgboost_optimal_config.txt")
    
    return optimizer, best_params

    return optimizer, best_params

# ============================================================================
# ANALYSIS AND VISUALIZATION
# ============================================================================

def plot_optimization_progress(optimizer, filename='online_xgboost_progress.png'):
    """Plot optimization progress"""
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    # Separate samples by fidelity
    low_samples = [s for s in optimizer.samples if s['fidelity'] == 'low']
    high_samples = [s for s in optimizer.samples if s['fidelity'] == 'high']
    
    low_losses = [s['loss'] for s in low_samples]
    high_losses = [s['loss'] for s in high_samples]
    
    low_valid = np.array([l for l in low_losses if np.isfinite(l)])
    high_valid = np.array([l for l in high_losses if np.isfinite(l)])
    
    # Plot 1: Loss over iterations
    ax = axes[0, 0]
    iterations = np.arange(len(optimizer.samples))
    
    low_iter = [i for i, s in enumerate(optimizer.samples) if s['fidelity'] == 'low']
    high_iter = [i for i, s in enumerate(optimizer.samples) if s['fidelity'] == 'high']
    
    ax.scatter(low_iter, low_losses, c='blue', alpha=0.6, s=50, label='Low-fidelity (30d)')
    ax.scatter(high_iter, high_losses, c='red', alpha=0.8, s=100, marker='s', 
               label='High-fidelity (180d)', edgecolors='black', linewidth=1.5)
    
    if optimizer.best_loss < np.inf:
        ax.axhline(optimizer.best_loss, color='green', linestyle='--', linewidth=2,
                  label=f'Best: {optimizer.best_loss:.4f}')
    
    ax.set_xlabel('Iteration', fontsize=12)
    ax.set_ylabel('Loss', fontsize=12)
    ax.set_title('Loss vs Iteration (Online Multi-Fidelity)', fontsize=13, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # Plot 2: Cumulative best
    ax = axes[0, 1]
    best_so_far = []
    current_best = np.inf
    for s in optimizer.samples:
        if s['fidelity'] == 'high' and np.isfinite(s['loss']) and s['loss'] < current_best:
            current_best = s['loss']
        best_so_far.append(current_best if current_best < np.inf else np.nan)
    
    valid_best = np.isfinite(best_so_far)
    if np.any(valid_best):
        ax.plot(iterations[valid_best], np.array(best_so_far)[valid_best], 
                'g-', linewidth=3, label='Best High-Fidelity')
    
    ax.set_xlabel('Iteration', fontsize=12)
    ax.set_ylabel('Best Loss', fontsize=12)
    ax.set_title('Convergence (High-Fidelity Only)', fontsize=13, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # Plot 3: Correlation scatter (low vs high)
    ax = axes[0, 2]
    
    # Find common points
    common_low = []
    common_high = []
    for s_high in high_samples:
        if not np.isfinite(s_high['loss']):
            continue
        for s_low in low_samples:
            if np.isfinite(s_low['loss']) and np.allclose(s_high['params'], s_low['params']):
                common_low.append(s_low['loss'])
                common_high.append(s_high['loss'])
                break
    
    if len(common_low) > 0:
        ax.scatter(common_low, common_high, s=80, alpha=0.7, edgecolors='black', linewidth=1)
        
        # Fit line
        if len(common_low) >= 3:
            z = np.polyfit(common_low, common_high, 1)
            p = np.poly1d(z)
            x_line = np.linspace(min(common_low), max(common_low), 100)
            ax.plot(x_line, p(x_line), 'r--', linewidth=2, label=f'y={z[0]:.2f}x+{z[1]:.2f}')
            
            # R²
            corr = np.corrcoef(common_low, common_high)[0, 1]
            ax.text(0.05, 0.95, f'R²={corr**2:.3f}\nn={len(common_low)}', 
                   transform=ax.transAxes, fontsize=10, verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        
        ax.plot([min(common_low), max(common_high)], [min(common_low), max(common_high)], 
               'k:', alpha=0.5, label='y=x')
        ax.set_xlabel('Low-Fidelity Loss (30 days)', fontsize=11)
        ax.set_ylabel('High-Fidelity Loss (180 days)', fontsize=11)
        ax.set_title('Fidelity Correlation', fontsize=13, fontweight='bold')
        ax.legend(fontsize=9)
        ax.grid(True, alpha=0.3)
    else:
        ax.text(0.5, 0.5, 'No common points yet', ha='center', va='center',
               transform=ax.transAxes, fontsize=12)
        ax.axis('off')
    
    # Plot 4: Loss distributions
    ax = axes[1, 0]
    if len(low_valid) > 0 and len(high_valid) > 0:
        bins = np.linspace(min(low_valid.min(), high_valid.min()),
                          max(low_valid.max(), high_valid.max()), 20)
        ax.hist(low_valid, bins=bins, alpha=0.6, label=f'Low (n={len(low_valid)})', color='blue')
        ax.hist(high_valid, bins=bins, alpha=0.6, label=f'High (n={len(high_valid)})', color='red')
        ax.axvline(optimizer.best_loss, color='green', linestyle='--', linewidth=2, label='Best')
        ax.set_xlabel('Loss', fontsize=11)
        ax.set_ylabel('Count', fontsize=11)
        ax.set_title('Loss Distributions', fontsize=13, fontweight='bold')
        ax.legend(fontsize=10)
        ax.grid(True, alpha=0.3)
    
    # Plot 5: Parameter evolution (best parameter)
    ax = axes[1, 1]
    if len(optimizer.samples) > 0:
        # Plot most sensitive parameter
        X_all = np.array([s['params'] for s in optimizer.samples])
        
        # Choose first parameter to show
        param_idx = 0
        param_name = PARAM_NAMES[param_idx]
        
        colors = ['blue' if s['fidelity'] == 'low' else 'red' for s in optimizer.samples]
        ax.scatter(iterations, X_all[:, param_idx], c=colors, alpha=0.6, s=50)
        
        if optimizer.best_params is not None:
            ax.axhline(optimizer.best_params[param_idx], color='green', linestyle='--', 
                      linewidth=2, label=f'Best: {optimizer.best_params[param_idx]:.4e}')
        
        ax.set_xlabel('Iteration', fontsize=11)
        ax.set_ylabel(param_name, fontsize=11)
        ax.set_title(f'{param_name} Evolution', fontsize=13, fontweight='bold')
        ax.legend(fontsize=9)
        ax.grid(True, alpha=0.3)
    
    # Plot 6: Computational savings
    ax = axes[1, 2]
    n_low = len(low_samples)
    n_high = len(high_samples)
    
    cost_actual = n_low / 6 + n_high  # 30-day is 1/6 cost of 180-day
    cost_baseline = len(optimizer.samples)  # If all were 180-day
    savings_pct = (1 - cost_actual / cost_baseline) * 100
    
    categories = ['Baseline\n(All 180d)', 'Multi-Fidelity\n(Mixed)']
    costs = [cost_baseline, cost_actual]
    colors_bar = ['gray', 'green']
    
    bars = ax.bar(categories, costs, color=colors_bar, alpha=0.7, edgecolor='black', linewidth=2)
    
    # Add values on bars
    for bar, cost in zip(bars, costs):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{cost:.1f}x', ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    ax.set_ylabel('Computational Cost\n(relative to single 180-day run)', fontsize=11)
    ax.set_title(f'Computational Savings: {savings_pct:.1f}%', fontsize=13, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    # Add breakdown text
    ax.text(0.5, 0.7, f'{n_low} × 30d + {n_high} × 180d\n= {n_low}/6 + {n_high} = {cost_actual:.1f}x',
           transform=ax.transAxes, fontsize=10, ha='center',
           bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
    
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"\n✓ Saved: {filename}")
    plt.close()

def export_results(optimizer, filename='online_xgboost_results.pkl'):
    """Export detailed results"""
    
    results = {
        'metadata': {
            'n_total_iterations': len(optimizer.samples),
            'n_low_fidelity': len([s for s in optimizer.samples if s['fidelity'] == 'low']),
            'n_high_fidelity': len([s for s in optimizer.samples if s['fidelity'] == 'high']),
            'n_successful_low': len([s for s in optimizer.samples if s['fidelity'] == 'low' and np.isfinite(s['loss'])]),
            'n_successful_high': len([s for s in optimizer.samples if s['fidelity'] == 'high' and np.isfinite(s['loss'])]),
            'best_loss': optimizer.best_loss,
            'best_iteration': optimizer.best_iteration,
            'parameter_names': PARAM_NAMES,
            'bounds': optimizer.bounds_dict,
        },
        'all_iterations': [],
        'best_result': None,
    }
    
    # Store all iterations
    for i, sample in enumerate(optimizer.samples):
        iter_data = {
            'iteration': i,
            'parameters': {name: float(sample['params'][j]) for j, name in enumerate(PARAM_NAMES)},
            'loss': float(sample['loss']) if np.isfinite(sample['loss']) else None,
            'fidelity': sample['fidelity'],
            'is_valid': bool(np.isfinite(sample['loss'])),
            'detailed_outputs': sample['detailed'],
        }
        results['all_iterations'].append(iter_data)
    
    # Store best result
    if optimizer.best_params is not None:
        results['best_result'] = {
            'iteration': optimizer.best_iteration,
            'parameters': {name: float(optimizer.best_params[j]) for j, name in enumerate(PARAM_NAMES)},
            'loss': optimizer.best_loss,
            'fidelity': optimizer.samples[optimizer.best_iteration]['fidelity'],
            'detailed_outputs': optimizer.samples[optimizer.best_iteration]['detailed'],
        }
    
    with open(filename, 'wb') as f:
        pickle.dump(results, f)
    
    print(f"✓ Exported detailed results: {filename}")
    
    # Print summary
    print("\n" + "="*70)
    print("RESULTS SUMMARY")
    print("="*70)
    print(f"\nTotal iterations: {results['metadata']['n_total_iterations']}")
    low_str = Colors.cyan(f"{results['metadata']['n_successful_low']}/{results['metadata']['n_low_fidelity']}")
    high_str = Colors.cyan(f"{results['metadata']['n_successful_high']}/{results['metadata']['n_high_fidelity']}")
    print(f"  Low-fidelity (30d):  {low_str} successful")
    print(f"  High-fidelity (180d): {high_str} successful")
    iter_str = Colors.blue(f"(iteration {results['metadata']['best_iteration'] + 1})")
    print(f"\n{Colors.bold('Best result')} {iter_str}:")
    loss_str = Colors.green(f"{results['metadata']['best_loss']:.6f}")
    print(f"  {Colors.bold('Loss:')} {loss_str}")
    
    if results['best_result'] and results['best_result']['detailed_outputs']:
        det = results['best_result']['detailed_outputs']
        q_str = Colors.cyan(f"{det['loss_q_bt']:.6f}")
        psi_str = Colors.cyan(f"{det['loss_psi_bt']:.6f}")
        print(f"  Q_bt NRMSE:   {q_str}")
        print(f"  Psi_bt NRMSE: {psi_str}")

if __name__ == "__main__":
    # Run optimization with hybrid strategy (150 iterations)
    optimizer, best_params = main(max_iterations=200)
    
    # Plot progress
    plot_optimization_progress(optimizer)
    
    # Export results
    export_results(optimizer)


HYBRID XGBOOST OPTIMIZATION

Strategy:
  • Iterations 1-50:  Multi-fidelity (30d + 180d mixed)
  • Iterations 51+:   Pure high-fidelity (all 180d)

✓ Loaded high-res: 512x256, 180 days
  Multi-fidelity comparison strategy:
    • Low-fidelity (30d):  compares to FIRST 30 days of high-res (spinup)
    • High-fidelity (180d): compares to LAST 30 days of high-res (equilibrium)
  This ensures fair temporal matching at each fidelity level

✓ Checkpoint found
✓ Loaded checkpoint:
  Total iterations: 163
  Low-fidelity:  135
  High-fidelity: 28
  Best loss: 0.439650

HYBRID XGBOOST OPTIMIZATION
Strategy:
  Phase 1 (iter 1-50):  Multi-fidelity exploration
                        - Low-fidelity (30d) for broad search
                        - Periodic high-fidelity (180d) verification
  Phase 2 (iter 51+):   Pure high-fidelity refinement
                        - ALL runs at 180 days
                        - Focus on finding true optimum
  Max iterations: 200

ITERATION 164/200
[PHASE 2: High-

100%|██████████| 8640/8640 [00:27<00:00, 315.55it/s]



LowRes_64x32 Simulation Complete!
  Comparing: Low-res last 30 days vs High-res last 30 days (equilibrium)
    High-res: 61 snapshots
    Low-res:  61 snapshots
  Loss: 1.694053 (averaged over 30 days)

  Status:
    Low-fidelity:  135/135 successful
    High-fidelity: 29/29 successful
    Best loss: 0.439650 (iteration 25)
  ✓ Progress saved

ITERATION 165/200
[PHASE 2: High-Fidelity Refinement]
Decision: HIGH-fidelity (pure_high_fidelity_phase)
  ✓ Low-fidelity model fitted on 135 samples
  ✓ High-fidelity model fitted on 29 samples
  ✓ Correlation model fitted on 17 common points

Testing parameters [HIGH-fidelity: 180 days]:
  viscosity_scale: 8.108147e-01
  drag_scale: 1.379291e+00
  eddy_diffusivity: 1.005549e+04
  smagorinsky_coeff: 3.311661e-02
  energy_correction: 3.304285e-03
  enstrophy_correction: 1.130831e-10

Running LowRes_64x32 Simulation
Grid: 64 x 32
Resolution: 31.2 km per grid point

Subgrid Parameters:
  viscosity_scale: 0.8108146679495691
  drag_scale: 1.37929064

100%|██████████| 8640/8640 [00:27<00:00, 317.64it/s]



LowRes_64x32 Simulation Complete!
  Comparing: Low-res last 30 days vs High-res last 30 days (equilibrium)
    High-res: 61 snapshots
    Low-res:  61 snapshots
  Loss: 0.951864 (averaged over 30 days)

  Status:
    Low-fidelity:  135/135 successful
    High-fidelity: 30/30 successful
    Best loss: 0.439650 (iteration 25)
  ✓ Progress saved

ITERATION 166/200
[PHASE 2: High-Fidelity Refinement]
Decision: HIGH-fidelity (pure_high_fidelity_phase)
  ✓ Low-fidelity model fitted on 135 samples
  ✓ High-fidelity model fitted on 30 samples
  ✓ Correlation model fitted on 17 common points

Testing parameters [HIGH-fidelity: 180 days]:
  viscosity_scale: 8.108147e-01
  drag_scale: 1.379291e+00
  eddy_diffusivity: 1.005549e+04
  smagorinsky_coeff: 3.311661e-02
  energy_correction: 3.304285e-03
  enstrophy_correction: 1.130831e-10

Running LowRes_64x32 Simulation
Grid: 64 x 32
Resolution: 31.2 km per grid point

Subgrid Parameters:
  viscosity_scale: 0.8108146679495691
  drag_scale: 1.37929064

...


LowRes_64x32 Simulation Complete!
  Comparing: Low-res last 30 days vs High-res last 30 days (equilibrium)
    High-res: 61 snapshots
    Low-res:  61 snapshots
  Loss: 0.951864 (averaged over 30 days)

  Status:
    Low-fidelity:  135/135 successful
    High-fidelity: 63/63 successful
    Best loss: 0.439650 (iteration 25)
  ✓ Progress saved

ITERATION 199/200
[PHASE 2: High-Fidelity Refinement]
Decision: HIGH-fidelity (pure_high_fidelity_phase)
  ✓ Low-fidelity model fitted on 135 samples
  ✓ High-fidelity model fitted on 63 samples
  ✓ Correlation model fitted on 17 common points

Testing parameters [HIGH-fidelity: 180 days]:
  viscosity_scale: 8.108147e-01
  drag_scale: 1.379291e+00
  eddy_diffusivity: 1.005549e+04
  smagorinsky_coeff: 3.311661e-02
  energy_correction: 3.304285e-03
  enstrophy_correction: 1.130831e-10

Running LowRes_64x32 Simulation
Grid: 64 x 32
Resolution: 31.2 km per grid point

Subgrid Parameters:
  viscosity_scale: 0.8108146679495691
  drag_scale: 1.37929064

100%|██████████| 8640/8640 [00:27<00:00, 316.75it/s]



LowRes_64x32 Simulation Complete!
  Comparing: Low-res last 30 days vs High-res last 30 days (equilibrium)
    High-res: 61 snapshots
    Low-res:  61 snapshots
  Loss: 0.951864 (averaged over 30 days)

  Status:
    Low-fidelity:  135/135 successful
    High-fidelity: 64/64 successful
    Best loss: 0.439650 (iteration 25)
  ✓ Progress saved

ITERATION 200/200
[PHASE 2: High-Fidelity Refinement]
Decision: HIGH-fidelity (pure_high_fidelity_phase)
  ✓ Low-fidelity model fitted on 135 samples
  ✓ High-fidelity model fitted on 64 samples
  ✓ Correlation model fitted on 17 common points

Testing parameters [HIGH-fidelity: 180 days]:
  viscosity_scale: 8.108147e-01
  drag_scale: 1.379291e+00
  eddy_diffusivity: 1.005549e+04
  smagorinsky_coeff: 3.311661e-02
  energy_correction: 3.304285e-03
  enstrophy_correction: 1.130831e-10

Running LowRes_64x32 Simulation
Grid: 64 x 32
Resolution: 31.2 km per grid point

Subgrid Parameters:
  viscosity_scale: 0.8108146679495691
  drag_scale: 1.37929064

100%|██████████| 8640/8640 [00:27<00:00, 318.52it/s]



LowRes_64x32 Simulation Complete!
  Comparing: Low-res last 30 days vs High-res last 30 days (equilibrium)
    High-res: 61 snapshots
    Low-res:  61 snapshots
  Loss: 0.951864 (averaged over 30 days)

  Status:
    Low-fidelity:  135/135 successful
    High-fidelity: 65/65 successful
    Best loss: 0.439650 (iteration 25)
  ✓ Progress saved

OPTIMIZATION COMPLETE

Total iterations: 200
  Low-fidelity (30 days):  135
  High-fidelity (180 days): 65
  Computational cost: ~87.5x vs 200x baseline
  Savings: 56.2%

Best loss: 0.439650
Best parameters:
  viscosity_scale: 2.055692e+00
  drag_scale: 5.284424e-01
  eddy_diffusivity: 3.420821e+04
  smagorinsky_coeff: 2.304021e-01
  energy_correction: 1.493304e-03
  enstrophy_correction: 1.409770e-10

✓ Saved: online_xgboost_optimal_params.pkl
✓ Saved: online_xgboost_optimal_config.txt

✓ Saved: online_xgboost_progress.png
✓ Exported detailed results: online_xgboost_results.pkl

RESULTS SUMMARY

Total iterations: 200
  Low-fidelity (30d):  135/